In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import loguniform
import matplotlib.pyplot as plt
from preprocessing import *
from save_and_compare_results import *
import shap

In [ ]:
X = pd.read_pickle("X_matrix_proteins_nb.pkl")
Y = pd.read_pickle("Y_matrix_proteins_nb.pkl")

# Preprocess the data
X = X.drop(columns=["Yeast_ID"]).fillna(0)
Y = Y.drop(columns=["Yeast_ID"]).fillna(Y.drop(columns=["Yeast_ID"]).mean())

In [ ]:
param_distributions = {
    "C": loguniform(1e-3, 1e3),
    "epsilon": loguniform(1e-3, 1),
    "gamma": ["scale", "auto"],
    "kernel": ["linear", "rbf"]
}

svr = SVR()
multi_svr = MultiOutputRegressor(RandomizedSearchCV(
    estimator=svr,
    param_distributions=param_distributions,
    n_iter=50, 
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
))

multi_svr.fit(X, Y)

In [ ]:
best_model = multi_svr.best_estimator_
best_params = multi_svr.best_params_
print("\nBest hyperparameters :", best_params)

In [ ]:
y_pred = best_model.predict(X)
r2 = r2_score(Y, y_pred)
mse = mean_squared_error(Y, y_pred)
print(f"\nR² Score: {r2:.4f}, Mean Squared Error: {mse:.4f}")

# Model features importance

In [ ]:
# Save feature importance from SVM
print("Saving SVM feature importances...")
save_feature_importance(
    features=X.columns,
    importance_scores=best_model.feature_importances_,
    method="model",
    model_name="SVM"
)

# Display and plot the top 10 features
print("Displaying top 10 features...")
feature_importances_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": best_model.feature_importances_
}).sort_values(by="Importance", ascending=False)

top_features = feature_importances_df.head(10)

print("\nTop 10 Features Impacting YPD Doubling Time:")
print(top_features)

plt.figure(figsize=(10, 6))
plt.barh(top_features["Feature"], top_features["Importance"], color="skyblue")
plt.xlabel("Coefficient Value")
plt.ylabel("Feature")
plt.title("Top 10 Features (RF)")
plt.gca().invert_yaxis()
plt.show()

# SHAP features importance

In [ ]:
print("Calcul of SHAP values...")
explainer = shap.Explainer(best_model, X)
shap_values = explainer(X)
shap_mean_importance = np.abs(shap_values.values).mean(axis=0)

# Sauvegarder les importances SHAP
print("Saving SHAP feature importances for SVM...")
save_feature_importance(
    features=X.columns,
    importance_scores=shap_mean_importance,
    method="SHAP",
    model_name="SVM"
)

# %%
# Graphiques SHAP
print("Generating SHAP summary plots for RF...")
shap.summary_plot(shap_values, X, plot_type="bar")
shap.summary_plot(shap_values, X)